## Deliverable 2. Create a Customer Travel Destinations Map.

In [48]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [11]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Deputatskiy,RU,69.30,139.90,36.91,89,90,2.57,overcast clouds
1,1,San Policarpo,PH,12.18,125.51,80.10,84,100,7.27,overcast clouds
2,2,Asau,RO,46.43,26.40,68.00,56,47,3.36,light rain
3,3,Jamestown,US,42.10,-79.24,78.01,78,75,10.29,broken clouds
4,4,Airai,TL,-8.93,125.41,56.35,91,49,4.07,scattered clouds


In [12]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [13]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                      (city_data_df["Max Temp"] <= max_temp)]

In [16]:
# Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [45]:
#Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,San Policarpo,PH,80.10,overcast clouds,12.18,125.51,
3,Jamestown,US,78.01,broken clouds,42.10,-79.24,
8,Seoul,KR,78.80,few clouds,37.57,126.98,
10,Albany,US,75.00,broken clouds,42.60,-73.97,
18,Butaritari,KI,82.35,overcast clouds,3.07,172.79,
25,Vila Franca Do Campo,PT,75.20,few clouds,37.72,-25.43,
27,Bengkulu,ID,75.51,overcast clouds,-3.80,102.27,
46,Arraial Do Cabo,BR,75.20,clear sky,-22.97,-42.03,
49,Tumbagaan,PH,81.00,broken clouds,5.37,120.31,
50,Vaini,TO,75.20,broken clouds,-21.20,-175.20,


In [46]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row["Lat"]
    lng =row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [51]:
# Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace("",np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,San Policarpo,PH,80.10,overcast clouds,12.18,125.51,Zal-Ann Beach Resort
3,Jamestown,US,78.01,broken clouds,42.10,-79.24,Hampton Inn & Suites Jamestown
8,Seoul,KR,78.80,few clouds,37.57,126.98,LOTTE HOTEL SEOUL
18,Butaritari,KI,82.35,overcast clouds,3.07,172.79,Isles Sunset Lodge
25,Vila Franca Do Campo,PT,75.20,few clouds,37.72,-25.43,Pestana Bahia Praia
...,...,...,...,...,...,...,...
694,Lucea,JM,89.60,few clouds,18.45,-78.17,Grand Palladium Jamaica Resort & Spa
696,Bandarbeyla,SO,75.94,clear sky,9.49,50.81,REER XOOSHLE GROUP
697,Kijang,KR,78.80,broken clouds,35.24,129.21,Brown Dot Gijang Yeonhwari
699,Garowe,SO,75.97,scattered clouds,8.41,48.48,Curubo Hotel


In [64]:
# Create the output File (CSV)
output_data_file = "../vacation_search/WeatherPy_vacation.csv"
# Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [67]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [69]:
# Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer =  gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))